In [1]:
import tweepy
import pandas as pd
import nltk
import string
import os
import re
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import udf
consumer_key = "Ax79DI6gXdMjOCQyQ61v2Khbj"
consumer_secret = "DBSmfUL50FlhpVnePa373lOpBFffJjrGInXbSGG4T7Ze2RRQTl"
access_token = "1418173405988376579-9g3SF5klnWOgd62nUfTfaqrQdzkJRA"
access_token_secret = "q3jO28xbGDlvE6djCNeoSMQbtkosZYT7mBTv0O6wznMpw"
# Creating the authentication object
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# Setting your access token and secret
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)


In [2]:
spark = SparkSession.builder.appName('classifier').getOrCreate()
df = spark.read.csv("trainingandtestdata/training.1600000.processed.noemoticon.csv")
df = df.withColumnRenamed("_c0", 'polarity')
df = df.withColumnRenamed("_c1", 'tweet_id')
df = df.withColumnRenamed("_c2", 'tweet_date')
df = df.withColumnRenamed("_c3", 'query_string')
df = df.withColumnRenamed("_c4", 'tweet_screen_name')
df = df.withColumnRenamed("_c5", 'Tweet')
df2=spark.read.csv("trainingandtestdata/testdata.manual.2009.06.14.csv")
df2 = df2.withColumnRenamed("_c0", 'polarity')
df2 = df2.withColumnRenamed("_c1", 'tweet_id')
df2 = df2.withColumnRenamed("_c2", 'tweet_date')
df2 = df2.withColumnRenamed("_c3", 'query_string')
df2 = df2.withColumnRenamed("_c4", 'tweet_screen_name')
df2 = df2.withColumnRenamed("_c5", 'Tweet')
df.show(5)
df2.show(5)

+--------+----------+--------------------+------------+-----------------+--------------------+
|polarity|  tweet_id|          tweet_date|query_string|tweet_screen_name|               Tweet|
+--------+----------+--------------------+------------+-----------------+--------------------+
|       0|1467810369|Mon Apr 06 22:19:...|    NO_QUERY|  _TheSpecialOne_|@switchfoot http:...|
|       0|1467810672|Mon Apr 06 22:19:...|    NO_QUERY|    scotthamilton|is upset that he ...|
|       0|1467810917|Mon Apr 06 22:19:...|    NO_QUERY|         mattycus|@Kenichan I dived...|
|       0|1467811184|Mon Apr 06 22:19:...|    NO_QUERY|          ElleCTF|my whole body fee...|
|       0|1467811193|Mon Apr 06 22:19:...|    NO_QUERY|           Karoli|@nationwideclass ...|
+--------+----------+--------------------+------------+-----------------+--------------------+
only showing top 5 rows

+--------+--------+--------------------+------------+-----------------+--------------------+
|polarity|tweet_id|        

In [3]:
df=df[['tweet_id','Tweet','polarity']]
df=df.withColumn('pre_clean_len',F.length('Tweet'))
df2=df2[['tweet_id','Tweet','polarity']]
df2=df2.withColumn('pre_clean_len',F.length('Tweet'))
df.groupBy('polarity').count().orderBy('count',ascending=False).show(10,False)
#df=df.toPandas()
#df.head(5)

+--------+------+
|polarity|count |
+--------+------+
|0       |800000|
|4       |800000|
+--------+------+



In [4]:
from nltk.corpus.reader.wordnet import *
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.tokenize import word_tokenize
from pyspark.sql.types import StringType,DoubleType,IntegerType
wn = nltk.WordNetLemmatizer()
worddict = set(nltk.corpus.words.words())

def preprocessing(text):
    wordset_n = set(wn.lemmatize(w, NOUN) for w in word_tokenize(text.lower().strip()))
    wordset_v = set(wn.lemmatize(w, VERB) for w in wordset_n)
    wordset = set(wn.lemmatize(w, ADJ) for w in wordset_v)
    wordset = wordset & worddict
    return ' '.join(list(wordset))
brand_udf=udf(preprocessing,StringType())
df=df.withColumn('text',brand_udf(df['Tweet']))
df2=df2.withColumn('text',brand_udf(df2['Tweet']))
df.show(5)
df2.show(5)

+----------+--------------------+--------+-------------+--------------------+
|  tweet_id|               Tweet|polarity|pre_clean_len|                text|
+----------+--------------------+--------+-------------+--------------------+
|1467810369|@switchfoot http:...|       0|          115|get day that thir...|
|1467810672|is upset that he ...|       0|          111|result cry school...|
|1467810917|@Kenichan I dived...|       0|           89|out ball save go ...|
|1467811184|my whole body fee...|       0|           47|whole feel my bod...|
|1467811193|@nationwideclass ...|       0|          111|at mad no all i c...|
+----------+--------------------+--------+-------------+--------------------+
only showing top 5 rows

+--------+--------------------+--------+-------------+--------------------+
|tweet_id|               Tweet|polarity|pre_clean_len|                text|
+--------+--------------------+--------+-------------+--------------------+
|       3|@stellargirl I lo...|       4|     

In [5]:
# Follow every follower of the authenticated user
timeline = api.home_timeline()
data = pd.DataFrame(data=[tweet.text for tweet in timeline], columns=['Tweet'])